In [17]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import spacy
import string
import gensim
import operator
import re

In [18]:
df_Product = pd.read_csv('marketing_sample_for_amazon_com-ecommerce_products_details__20190601_20190630__30k_data.csv')

df_Product['Title_Description']=df_Product.apply(lambda x:'%s , %s' % (x['Title'],x['Description']),axis=1)

df_Product.head()

,Uniq Id,Crawl Timestamp,Asin,Title,Image Url,Weight,Description,Price,Buybox Winner,Other Seller1,Other Seller1 Price,Other Seller2,Other Seller2 Price,Other Seller3,Other Seller3 Price,Category,Available,Title_Description
0,6f1d3625f8ee281d98f0dff8ce4a6e85,2019-06-05 07:20:52 +0000,B00F0OIZZI,"Pilot Needle Ballpoint Pen, Hi-Tec V10, Blue ...",https://images-na.ssl-images-amazon.com/images...,0.32 ounces,NaN,NaN,"Haniwa port. Reviews P.when(""seller-register-p...",Japanese poppin culture,4.09,Amazon.com,8.02,NaN,NaN,Office Products › Office & School Supplies › ...,Available,"Pilot Needle Ballpoint Pen, Hi-Tec V10, Blue ..."
1,6145cce0623a36e3ca2691ebda1da9b4,2019-06-05 09:50:42 +0000,B007NZEL14,Swissmar Capstore Select Storage Rack for 18-...,https://images-na.ssl-images-amazon.com/images...,1.11 pounds,Swissmar's capstore select 18 storage unit kee...,NaN,"Comfort House. Reviews P.when(""seller-register...",NaN,NaN,NaN,NaN,NaN,NaN,Home & Kitchen Kitchen & Dining Kitchen Utens...,NaN,Swissmar Capstore Select Storage Rack for 18-...
2,be4e1e8b9d9af09095b92ef2c00271f6,2019-06-05 01:43:08 +0000,B00ZW10EQ2,Gemini200 Delta CV-880 Gold Crown Livery Airc...,https://images-na.ssl-images-amazon.com/images...,1.6 pounds,Welcome to the exciting world of GeminiJets! O...,NaN,"Aiken's Airplanes. Reviews P.when(""seller-regi...",NaN,NaN,NaN,NaN,NaN,NaN,Toys & Games Hobbies Models & Model Kits Pre-...,Available,Gemini200 Delta CV-880 Gold Crown Livery Airc...
3,c8514f6b82a86a2bdc814feba8953c28,2019-06-03 00:27:22 +0000,1450,Beauty & Personal Care Makeup Eyes Eyeliner,0.32 ounces,B017OE9QLI,Kat Von D Tattoo Liquid Eye Liner in Trooper ...,https://images-na.ssl-images-amazon.com/images...,"Achieve eye perfection with this high-pigment,...",NaN,Cocodrilo Deals,14.50,LUXBEAUTY,16.40,NaN,NaN,NaN,Beauty & Personal Care Makeup Eyes Eyeliner ...
4,867f548e26d2453386de0e953b4442a6,2019-06-03 05:50:00 +0000,493,Toys & Games Party Supplies Party Tableware N...,1 pounds,B072LPH8MG,American Greetings DC Super Hero Girls Lunch ...,https://images-na.ssl-images-amazon.com/images...,Calling all Super hero girls fans! these brigh...,Amazon.com.,A Wolfpack company,4.93,Sass & Salt,5.00,NaN,NaN,NaN,Toys & Games Party Supplies Party Tableware N...


In [19]:
from spacy.lang.en.stop_words import STOP_WORDS


In [20]:

spacy_nlp = spacy.load('en_core_web_sm')

#create list of punctuations and stopwords
punctuations = string.punctuation
stop_words = spacy.lang.en.stop_words.STOP_WORDS

#function for data cleaning and processing
#This can be further enhanced by adding / removing reg-exps as desired.

def spacy_tokenizer(sentence):
 
    #remove distracting single quotes
    sentence = re.sub('\'','',sentence)

    #remove digits adnd words containing digits
    sentence = re.sub('\w*\d\w*','',sentence)

    #replace extra spaces with single space
    sentence = re.sub(' +',' ',sentence)

    #remove unwanted lines starting from special charcters
    sentence = re.sub(r'\n: \'\'.*','',sentence)
    sentence = re.sub(r'\n!.*','',sentence)
    sentence = re.sub(r'^:\'\'.*','',sentence)
    
    #remove non-breaking new line characters
    sentence = re.sub(r'\n',' ',sentence)
    
    #remove punctunations
    sentence = re.sub(r'[^\w\s]',' ',sentence)
    
    #creating token object
    tokens = spacy_nlp(sentence)
    
    #lower, strip and lemmatize
    tokens = [word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in tokens]
    
    #remove stopwords, and exclude words less than 2 characters
    tokens = [word for word in tokens if word not in stop_words and word not in punctuations and len(word) > 2]
    
    #return tokens
    return tokens

In [21]:
print ('Cleaning and Tokenizing...')
%time df_Product['Title_tokenized'] = df_Product['Title_Description'].map(lambda x: spacy_tokenizer(x))

df_Product.head()

Cleaning and Tokenizing...
CPU times: total: 8min 23s
Wall time: 10min 21s


,Uniq Id,Crawl Timestamp,Asin,Title,Image Url,Weight,Description,Price,Buybox Winner,Other Seller1,Other Seller1 Price,Other Seller2,Other Seller2 Price,Other Seller3,Other Seller3 Price,Category,Available,Title_Description,Title_tokenized
0,6f1d3625f8ee281d98f0dff8ce4a6e85,2019-06-05 07:20:52 +0000,B00F0OIZZI,"Pilot Needle Ballpoint Pen, Hi-Tec V10, Blue ...",https://images-na.ssl-images-amazon.com/images...,0.32 ounces,NaN,NaN,"Haniwa port. Reviews P.when(""seller-register-p...",Japanese poppin culture,4.09,Amazon.com,8.02,NaN,NaN,Office Products › Office & School Supplies › ...,Available,"Pilot Needle Ballpoint Pen, Hi-Tec V10, Blue ...","[pilot, needle, ballpoint, pen, tec, blue, ink..."
1,6145cce0623a36e3ca2691ebda1da9b4,2019-06-05 09:50:42 +0000,B007NZEL14,Swissmar Capstore Select Storage Rack for 18-...,https://images-na.ssl-images-amazon.com/images...,1.11 pounds,Swissmar's capstore select 18 storage unit kee...,NaN,"Comfort House. Reviews P.when(""seller-register...",NaN,NaN,NaN,NaN,NaN,NaN,Home & Kitchen Kitchen & Dining Kitchen Utens...,NaN,Swissmar Capstore Select Storage Rack for 18-...,"[swissmar, capstore, select, storage, rack, pa..."
2,be4e1e8b9d9af09095b92ef2c00271f6,2019-06-05 01:43:08 +0000,B00ZW10EQ2,Gemini200 Delta CV-880 Gold Crown Livery Airc...,https://images-na.ssl-images-amazon.com/images...,1.6 pounds,Welcome to the exciting world of GeminiJets! O...,NaN,"Aiken's Airplanes. Reviews P.when(""seller-regi...",NaN,NaN,NaN,NaN,NaN,NaN,Toys & Games Hobbies Models & Model Kits Pre-...,Available,Gemini200 Delta CV-880 Gold Crown Livery Airc...,"[delta, gold, crown, livery, aircraft, scale, ..."
3,c8514f6b82a86a2bdc814feba8953c28,2019-06-03 00:27:22 +0000,1450,Beauty & Personal Care Makeup Eyes Eyeliner,0.32 ounces,B017OE9QLI,Kat Von D Tattoo Liquid Eye Liner in Trooper ...,https://images-na.ssl-images-amazon.com/images...,"Achieve eye perfection with this high-pigment,...",NaN,Cocodrilo Deals,14.50,LUXBEAUTY,16.40,NaN,NaN,NaN,Beauty & Personal Care Makeup Eyes Eyeliner ...,"[beauty, personal, care, makeup, eyes, eyeline..."
4,867f548e26d2453386de0e953b4442a6,2019-06-03 05:50:00 +0000,493,Toys & Games Party Supplies Party Tableware N...,1 pounds,B072LPH8MG,American Greetings DC Super Hero Girls Lunch ...,https://images-na.ssl-images-amazon.com/images...,Calling all Super hero girls fans! these brigh...,Amazon.com.,A Wolfpack company,4.93,Sass & Salt,5.00,NaN,NaN,NaN,Toys & Games Party Supplies Party Tableware N...,"[toy, games, party, supplies, party, tableware..."


In [22]:
product_plot = df_Product['Title_tokenized']
product_plot[0:5]

0    [pilot, needle, ballpoint, pen, tec, blue, ink...
1    [swissmar, capstore, select, storage, rack, pa...
2    [delta, gold, crown, livery, aircraft, scale, ...
3    [beauty, personal, care, makeup, eyes, eyeline...
4    [toy, games, party, supplies, party, tableware...
Name: Title_tokenized, dtype: object

In [23]:
from gensim import corpora

#creating term dictionary
%time dictionary = corpora.Dictionary(product_plot)

#filter out terms which occurs in less than 4 documents and more than 20% of the documents.
# NOTE: Since we have smaller dataset, we will keep this commented for now.

#dictionary.filter_extremes(no_below=4, no_above=0.2)

#list of few which which can be further removed
stoplist = set('hello and if this can would should could tell ask stop come go')
stop_ids = [dictionary.token2id[stopword] for stopword in stoplist if stopword in dictionary.token2id]
dictionary.filter_tokens(stop_ids)

CPU times: total: 2.02 s
Wall time: 2.18 s


In [24]:
#print top 50 items from the dictionary with their unique token-id
dict_tokens = [[[dictionary[key], dictionary.token2id[dictionary[key]]] for key, value in dictionary.items() if key <= 50]]
print (dict_tokens)

[[['ballpoint', 0], ['blue', 1], ['ink', 2], ['lhgn', 3], ['nan', 4], ['needle', 5], ['pen', 6], ['pilot', 7], ['tec', 8], ['able', 9], ['access', 10], ['allow', 11], ['apartment', 12], ['approximately', 13], ['available', 14], ['base', 15], ['blend', 16], ['bright', 17], ['capstore', 18], ['chrome', 19], ['clean', 20], ['college', 21], ['color', 22], ['cup', 23], ['decor', 24], ['design', 25], ['efficient', 26], ['frame', 27], ['gift', 28], ['great', 29], ['inch', 30], ['kitchen', 31], ['measure', 32], ['neat', 33], ['need', 34], ['organize', 35], ['pack', 36], ['polished', 37], ['pound', 38], ['rack', 39], ['reachable', 40], ['ready', 41], ['reign', 42], ['resident', 43], ['rotate', 44], ['select', 45], ['separately', 46], ['space', 47], ['stainless', 48], ['stand', 49], ['steel', 50]]]


In [25]:
corpus = [dictionary.doc2bow(desc) for desc in product_plot]

word_frequencies = [[(dictionary[id], frequency) for id, frequency in line] for line in corpus[0:3]]

print(word_frequencies)

[[('ballpoint', 1), ('blue', 1), ('ink', 1), ('lhgn', 1), ('nan', 1), ('needle', 1), ('pen', 1), ('pilot', 1), ('tec', 1)], [('able', 1), ('access', 1), ('allow', 1), ('apartment', 1), ('approximately', 1), ('available', 1), ('base', 1), ('blend', 1), ('bright', 1), ('capstore', 2), ('chrome', 1), ('clean', 1), ('college', 1), ('color', 1), ('cup', 3), ('decor', 1), ('design', 1), ('efficient', 1), ('frame', 1), ('gift', 1), ('great', 1), ('inch', 1), ('kitchen', 1), ('measure', 1), ('neat', 1), ('need', 1), ('organize', 1), ('pack', 4), ('polished', 1), ('pound', 1), ('rack', 1), ('reachable', 1), ('ready', 1), ('reign', 1), ('resident', 1), ('rotate', 1), ('select', 2), ('separately', 1), ('space', 2), ('stainless', 1), ('stand', 1), ('steel', 2), ('storage', 3), ('store', 1), ('student', 1), ('swissmar', 2), ('tall', 1), ('unit', 3), ('use', 1), ('weigh', 1)], [('aircraft', 1), ('collectible', 1), ('crown', 1), ('delta', 1), ('edition', 1), ('exciting', 1), ('geminijets', 1), ('gold

In [26]:
%time products_tfidf_model = gensim.models.TfidfModel(corpus, id2word=dictionary)
%time products_lsi_model = gensim.models.LsiModel(products_tfidf_model[corpus], id2word=dictionary, num_topics=300)

CPU times: total: 469 ms
Wall time: 477 ms
CPU times: total: 1min 10s
Wall time: 41.8 s


In [27]:
%time gensim.corpora.MmCorpus.serialize('products_tfidf_model_mm', products_tfidf_model[corpus])
%time gensim.corpora.MmCorpus.serialize('products_lsi_model_mm',products_lsi_model[products_tfidf_model[corpus]])

CPU times: total: 7.72 s
Wall time: 8.85 s
CPU times: total: 41.7 s
Wall time: 53.1 s


In [28]:
#Load the indexed corpus
products_tfidf_corpus = gensim.corpora.MmCorpus('products_tfidf_model_mm')
products_lsi_corpus = gensim.corpora.MmCorpus('products_lsi_model_mm')

print(products_tfidf_corpus)
print(products_lsi_corpus)

MmCorpus(30000 documents, 33814 features, 557308 non-zero entries)
MmCorpus(30000 documents, 300 features, 8999998 non-zero entries)


In [29]:
from gensim.similarities import MatrixSimilarity

%time products_index = MatrixSimilarity(products_lsi_corpus, num_features = products_lsi_corpus.num_terms)

CPU times: total: 27 s
Wall time: 30.4 s


In [30]:
from operator import itemgetter

In [31]:

def search_similar_products(search_term):

    query_bow = dictionary.doc2bow(spacy_tokenizer(search_term))
    query_tfidf = products_tfidf_model[query_bow]
    query_lsi = products_lsi_model[query_tfidf]

    products_index.num_best = 5

    productss_list = products_index[query_lsi]

    productss_list.sort(key=itemgetter(1), reverse=True)
    products_names = []

    for j, products in enumerate(productss_list):

        products_names.append (
            {
                'Relevance': round((products[1] * 100),2),
                'products Title': df_Product['Title'][products[0]],
                'Description': df_Product['Description'][products[0]]
            }

        )
        if j == (products_index.num_best-1):
            break

    return pd.DataFrame(products_names, columns=['Relevance','products Title','Description'])

In [32]:
# search for products tiles that are related to below search parameters
search_similar_products('Good Grips Meat Tenderizer')

,Relevance,products Title,Description
0,98.61,OXO 26191 Good Grips Meat Tenderizer,NaN
1,85.10,Good Cook Epicure Pasta Fork,NaN
2,84.01,OXO Good Grips Medium Coarse Grater,NaN
3,80.25,"OXO Good Grips Utensil Set, 3-Piece",NaN
4,80.15,Good Cook Touch Turner,NaN
